In [1]:
# Importing the StringIO module.
from bs4 import BeautifulSoup

# import the requests library 
import requests

# Importing the StringIO module.
from io import StringIO

import pandas as pd

import re
from tqdm import tqdm

#Inserting stock data into SQL database using sqlachemy
from sqlalchemy import create_engine

#Defining headers with User agent to establish requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

import pandas as pd
import numpy as np

In [2]:

#Function to scrape Marketsmith stock fundamental data
def marketsmith_scrape(stock_list):
    
    for x in tqdm(range(0,len(stock_list))):
        #time.sleep(2)
        options = Options()
        options.add_argument('--headless')

        #Firefox browser webdriver
        driver = webdriver.Firefox(options=options)

        #Google chrome webdriver
        #options = webdriver.ChromeOptions()
        #driver = webdriver.Chrome(options=options)
        
        #Base Url to fetch stock fundamental data through request
        url = "https://marketsmithindia.com/mstool/eval/{}/evaluation.jsp#/".format(stock_list[x])
        print(url)
        driver.get(url)
        #soup = BeautifulSoup(driver.page_source, 'lxml')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        

        #Populating stock ratings data for individual stocks from Marketsmith evaluation pages
        if x==0:
            stock_detail_header = soup.find_all('div', class_ = "details")
            
        cols = []
        
        #Generating columns through iteration for stock_detail_header
        
        for y in range(0,len(stock_detail_header)):
            cols.append(stock_detail_header[y].get_text().strip())
            
        #Creating a new column for stock wise segregation
        cols.insert(0,"Stock Name")
        
        print(y,"columns","--->>>",cols)
        
        #Generating row values for the stock_detail_header header 
        stock_detail_value = soup.find_all('div', class_ = "value")
        print(format(stock_list[x]),"StockDetailValue>>>",stock_detail_value) 
        

        #Feeding an empty Dataframe with its respective column values through iterating summary_data list
        #Creating an empty Dataframe to feed the column names through iterating summary_cols list

        summary_data_values = []
        

        #Generating columns data through iteration
        for z in range(0,len(stock_detail_value)):
            print(stock_detail_value[z].get_text().strip())
            summary_data_values.append(stock_detail_value[z].get_text().strip())
        
        #Checking if all data values were empty for a respective stock in Marketsmith by 
        #moving list into set to avoid duplicate values and see if it matches '' empty value
        if ('' in (set(summary_data_values))):
            print("########Stock Not PresentSummary Set values#########", set(summary_data_values))
            continue
        
        #Adding stock name as a new column to every data rows for identification.
        summary_data_values.insert(0,stock_list[x])       
        
        print(z,"summary_data_values","--->>>",summary_data_values)    
        
        #Creating base data frame to hold data for every stock
        print('*****Columns******',cols,'^^^^DATA^^^^^',[summary_data_values])
        share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
        
        #Removing duplicate rows to avoid discrepency
        share_summary = share_summary.loc[:,~share_summary.columns.duplicated()]
        
        #Creating final Data frame one time and from base dataframe structure
        if x==0:
            #share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
            
            all_share_summary = pd.DataFrame(columns= [share_summary])
        print ("Inside x loop",x)
            
        #Adding individual stock data values through every iteration    
        print("Final Datatypes","<<<<<<<<->>>>>>>>",all_share_summary.dtypes)
        all_share_summary = all_share_summary._append(share_summary,ignore_index=True)
        
        #del(stock_detail_header,stock_detail_value,share_summary)
        summary_data_values.clear()
              
        
        #Quarterly earnings header objects
            
        stock_detail_quarterly_headers = soup.find('table', class_ = "table table-condensed tableDetails", id = "formattedSalesAndEarningTable")
        #Filtering table header content from soup objects - stock_detail_quarterly_headers
        quarterly_objects = stock_detail_quarterly_headers.find_all('th')

        #Iterating quarterly earnings headers by element to add into list for further processing
        #Not iterating all rows because last rows is NULL
        stock_quarterly_earnings_headers = [ title.text.strip() for title in quarterly_objects ]
        stock_quarterly_earnings_headers.insert(0,"Stock Name")
        #Removing last empty values in list on every iteration
        stock_quarterly_earnings_headers.pop()
        
        
        print("Final Quarterly Earnings Headers","<<<<<>>>>>>",stock_quarterly_earnings_headers)
        
        print(format(stock_list[x]),"StockDetailHeader>>>" ,stock_detail_header)
        #print(format(lst[x]),"QuarterlyEPSHeaders>>>" , stock_detail_quarterly_headers)
        print(stock_detail_quarterly_headers.find_all('th'))
        
        #Filtering data contents for Quarterly earnings
        quarterly_objects_data = stock_detail_quarterly_headers.find_all('tr')
        
        #Creating final Data frame for quarterly earnings one time and from base dataframe structure
        if x==0:
            quarterly_earnings = pd.DataFrame(columns= stock_quarterly_earnings_headers)
            #quarterly_earnings.rename(columns= {'Stock Name': 'stock_name', 'Date(Transcript )': 'date' , 'EPS' : 'eps', \
            #                                    '%Chg': 'eps_chg_pct', 'Sales(Cr)' : 'sales_in_cr', '%Chg': 'sales_chg_pct'},inplace= True)
            print("Columns",quarterly_earnings.columns)
            
            quarterly_earnings_all_stocks = pd.DataFrame(columns= stock_quarterly_earnings_headers)
            
        print("earnings dataframe",stock_quarterly_earnings_headers)
        
        #Iterating individual quarterly earnings table row by element to add into list for further processing
        quarterly_earnings_lst = []
        for rows in quarterly_objects_data:
            row_data = rows.find_all('td')
            ind_quarterly_earnings_rows = [ q_data.text.strip() for q_data in row_data]
            #Adding Stock name to quarterly earnings rows manually
            ind_quarterly_earnings_rows.insert(0,stock_list[x])
            #Removing last empty values in list on every iteration
            ind_quarterly_earnings_rows.pop()
            
            #Adding iterated quarterly earnings data to list for a stock on their respective iteration of different quarters
            quarterly_earnings_lst.append(ind_quarterly_earnings_rows)
            
            print("ind_quarterly_earnings_rows_Final Quarterly Earnings Headers rows","<<<<<>>>>>>",ind_quarterly_earnings_rows)
        quarterly_earnings_lst.pop(0)
        print("Final List",">><><><><><><><><>",quarterly_earnings_lst)
        
        #Appending quarterly earnings rows stock specifically to dataframe 
        quarterly_earnings = pd.DataFrame(columns= stock_quarterly_earnings_headers, data = quarterly_earnings_lst)
        quarterly_earnings_all_stocks = quarterly_earnings_all_stocks._append(quarterly_earnings, ignore_index=True)
         
    
        print("QuarterlyEarnings Dataframe:",quarterly_earnings)
        

        

        #share_summary = pd.DataFrame(columns= cols)
        

        #Deleting objects cache to contents to store data in next iteration
        soup.decompose()
        
        #Closing the driver    
        driver.quit()
        print("Next Iteration")

    #Data cleaning - Dropping unncessary columns
    all_share_summary.drop(list(all_share_summary)[0:23], axis=1,inplace = True)
    
    #Renaming column name for SQL table processing
    
    all_share_summary.rename(columns = {'Stock Name' : 'stock_name', 'Market Capitalization' : 'market_capitalization', 'Sales' : 'sales', \
        'Shares in Float' : 'shares_in_float', 'No of Funds' : 'no_of_funds', 'Shares held by Funds' : 'shares_held_by_funds', 'Yield' : 'yield', \
        'Book Value' : 'book_value', 'U/D Vol Ratio' : 'u/d_vol_ratio', 'LTDebt/Equity' : 'ltdebt_equity', 'Alpha' : 'alpha', 'Beta' : 'beta', \
        'Master Score' : 'master_score', 'EPS Rating' : 'eps_rating', 'Price Strength' : 'price_strength', 'Acc/Dis Rating' : 'buyers_demand', \
        'Group Rank' : 'group_rank_out_of_197', 'EPS Growth Rate' : 'eps_growth_rate', 'Earnings Stability' : 'earnings_stability', 'P/E Ratio' : 'pe_ratio', \
        '5-Year P/E Range' : '5years_pe_range', 'Return on Equity' : 'return_on_equity', 'Cash Flow (INR)': 'cash_flow'},
        inplace = True)
    
    
    #Slicing EPS % Chg and Sales % Chg column as it is having same name so we have issue on processing it individually
    #To overcome that we have renamed that to map EPS & Sales % change meaningfully 
    eps_pct = quarterly_earnings_all_stocks.iloc[:,3]
    eps_pct.columns = ['eps_change_pct']
    print("Sliced w dataframe:",eps_pct.columns)
    print("Sliced w dataframe:",eps_pct)
    
    sales_pct = quarterly_earnings_all_stocks.iloc[:,5]
    sales_pct.columns = ['sales_change_pct']
    print("Sliced w dataframe:",sales_pct.columns)
    print("Sliced w dataframe:",sales_pct)
    
    #Dropping existing %Change column we will add later below with appropriate
    quarterly_earnings_all_stocks = quarterly_earnings_all_stocks.drop(columns = ['%Chg'])
       
    #Adding back the columns for EPS%Change and Sales%Change with right column names    
    quarterly_earnings_all_stocks['EPS_Change_%'] = eps_pct
    quarterly_earnings_all_stocks['SALES_Change_%'] = sales_pct
    
    
    #Reindexing dataframe in appropriate order
    quarterly_earnings_all_stocks = quarterly_earnings_all_stocks.reindex(['Stock Name','Date(Transcript )',\
                                    'EPS','EPS_Change_%','Sales(Cr)','SALES_Change_%'], axis = 1)
    
    
    
    #Renaming the column names appropriately in final dataframe before inserting into SQL
    quarterly_earnings_all_stocks.rename(columns={'Stock Name':'stock_name', 'Date(Transcript )': 'quarter', 'EPS':'eps', 'EPS_Change_%':'eps_pct_chg',\
                                                  'Sales(Cr)':'sales_in_cr', 'SALES_Change_%': 'sales_pct_chg'}, inplace= True)
    
    
    
    print("Final EPS DF columns","##########",quarterly_earnings_all_stocks.columns)
    print("Final EPS Dataframe","##########",quarterly_earnings_all_stocks)
        
    #Removing INR string (text cleansing) from few columns to fit into perfect standard for data manipulation
    all_share_summary.market_capitalization = all_share_summary.market_capitalization.str.replace('INR ','')
    all_share_summary.sales = all_share_summary.sales.str.replace('INR ','')
    all_share_summary.group_rank_out_of_197 = all_share_summary.group_rank_out_of_197.str.replace(' of 197','')
    
    
    #Changing datatypes for the columns to their appropriate type for further processing
    all_share_summary.master_score = all_share_summary.master_score.str.replace('N/A','0').astype('Int32')
    all_share_summary.eps_rating = all_share_summary.eps_rating.str.replace('N/A','0').astype('Int32')
    all_share_summary.price_strength = all_share_summary.price_strength.str.replace('N/A','0').astype('Int32')
    all_share_summary.earnings_stability = all_share_summary.earnings_stability.str.replace('N/A','0').astype('Int32')
    all_share_summary.pe_ratio = all_share_summary.pe_ratio.str.replace('N/A','0').astype('Int32')
    all_share_summary.group_rank_out_of_197 = all_share_summary.group_rank_out_of_197.str.replace('N/A','0').astype('Int32')

    
    #quarterly_earnings_all_stocks.eps = quarterly_earnings_all_stocks.eps.str.replace('','0').astype('float64')
    #quarterly_earnings_all_stocks.sales_in_cr = quarterly_earnings_all_stocks.sales_in_cr.str.replace('','0').astype('float64')
    #quarterly_earnings_all_stocks.sales_in_cr = quarterly_earnings_all_stocks.sales_in_cr.str.replace(',','').astype('float64')
    
    
    #Inserting stock data into SQL database using sqlachemy
    
    #Defining table name on MSSQL server to locate the data
    stock_eval_table_name = "market_smith_stock_eval_test"
    all_stocks_quarterly_earnings = "all_stocks_quarterly_earnings_test"
    sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')

    db_conn= sql_engine.connect()

    #DB Actions to load data from Pandas Dataframe to MSSQL
    try:
        all_share_summary.to_sql(stock_eval_table_name, db_conn, if_exists= 'append',index= False)
        quarterly_earnings_all_stocks.to_sql(all_stocks_quarterly_earnings, db_conn, if_exists= 'append',index= False)
        
    except Exception as ex:
        print(ex)
    else:
        print('Marketsmith Stock Evalulation data successfully inserted into MS SQL table-{}'.format(stock_eval_table_name))
        print('Marketsmith Stock Evalulation data successfully inserted into MS SQL table-{}'.format(all_stocks_quarterly_earnings))
    finally:
        db_conn.close()
    
    return all_share_summary


In [3]:

#Reading CSV file to fetch all listed Stocks from NSE website

all_equity_lst = pd.read_csv("./EQUITY_L.csv")

stock_list = []

for ind_stock in all_equity_lst['SYMBOL']:
    stock_list.append(ind_stock)

print("Inside Stock_List:",stock_list)

sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')
table_trun_conn = sql_engine.connect()
sql_engine.execute('delete from market_smith_stock_eval_test')
sql_engine.execute('delete from all_stocks_quarterly_earnings_test')
table_trun_conn.close()

#Below code part is used to iterate every set of 10 stocks 
counter = 0
top_stock = ''
while (counter < len(stock_list)):
    if(counter!=0):
        sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')
        db_conn= sql_engine.connect()
        top_stock = str(pd.read_sql_query(con = db_conn ,sql='select top 1 [stock_name] From market_smith_stock_eval_test order by stock_name desc'))
        db_conn.close()
    
#Calling webscraping function
    if(stock_list[counter] != top_stock):
        marketsmith_scrape(stock_list[counter:5+ counter])
        counter = counter + 5
        print('Next Iteration on WHILE LOOP')


Inside Stock_List: ['20MICRONS', '21STCENMGM', '360ONE', '3IINFOLTD', '3MINDIA', '3PLAND', '5PAISA', '63MOONS', 'A2ZINFRA', 'AAATECH', 'AAKASH', 'AAREYDRUGS', 'AARON', 'AARTECH', 'AARTIDRUGS', 'AARTIIND', 'AARTIPHARM', 'AARTISURF', 'AARVEEDEN', 'AARVI', 'AAVAS', 'ABAN', 'ABB', 'ABBOTINDIA', 'ABCAPITAL', 'ABFRL', 'ABMINTLLTD', 'ABSLAMC', 'ACC', 'ACCELYA', 'ACCURACY', 'ACE', 'ACEINTEG', 'ACI', 'ACL', 'ACLGATI', 'ADANIENSOL', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADFFOODS', 'ADL', 'ADORWELD', 'ADROITINFO', 'ADSL', 'ADVANIHOTR', 'ADVENZYMES', 'AEGISCHEM', 'AEROFLEX', 'AETHER', 'AFFLE', 'AGARIND', 'AGI', 'AGRITECH', 'AGROPHOS', 'AGSTRA', 'AHL', 'AHLADA', 'AHLEAST', 'AHLUCONT', 'AIAENG', 'AIIL', 'AIRAN', 'AIROLAM', 'AJANTPHARM', 'AJMERA', 'AJOONI', 'AKASH', 'AKG', 'AKI', 'AKSHAR', 'AKSHARCHEM', 'AKSHOPTFBR', 'AKZOINDIA', 'ALANKIT', 'ALBERTDAVD', 'ALEMBICLTD', 'ALICON', 'ALKALI', 'ALKEM', 'ALKYLAMINE', 'ALLCARGO', 'ALLSEC', 'ALMONDZ', 'ALOKINDS', 'ALPA', 'ALPHAGEO', 'ALPSINDU

  0%|          | 0/5 [00:00<?, ?it/s]

https://marketsmithindia.com/mstool/eval/20MICRONS/evaluation.jsp#/


  0%|          | 0/5 [02:06<?, ?it/s]


UnboundLocalError: cannot access local variable 'y' where it is not associated with a value

In [ ]:
#Reading CSV file to fetch all listed NSE Stocks from local folder
all_equity_lst = pd.read_csv("./NSE_EQUITY_List.csv")

stock_list = []

for ind_stock in all_equity_lst['SYMBOL']:
    stock_list.append(ind_stock)

print("Inside Stock_List:",stock_list)

#Inserting stock data into SQL database using sqlachemy
from sqlalchemy import create_engine

sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')

db_conn= sql_engine.connect()

sql_res = pd.read_sql_query(con = db_conn ,sql='select top 1 [stock_name] From market_smith_stock_eval order by stock_name desc')

print("Last Stock Data in Table",sql_res)

db_conn.close()

print("Index of Result",stock_list[1000])